In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import json

ModuleNotFoundError: No module named 'selenium'

In [28]:
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)

driver.get("https://www.sofascore.com/")
time.sleep(5)

groups = ['summary', 'attack', 'defence', 'passing', 'goalkeeper']

# go through 29 pages
for group in groups:
    all_results = []
    print(f"Scraping group: {group}")
    
    for page in range(1, 30):
        offset = (page - 1) * 20
        url = f"https://www.sofascore.com/api/v1/unique-tournament/17/season/52186/statistics?limit=20&order=-rating&offset={offset}&accumulation=total&group={group}"
        
        driver.get(url)
        time.sleep(3) 
        
        # extract JSON from source
        try:
            json_content = driver.find_element("tag name", 'pre').text
            data = json.loads(json_content)
            all_results.extend(data['results'])
            print(f"Page {page} scraped successfully.")
        except Exception as e:
            print(f"Error on page {page}: {str(e)}")
            # Save page source for debugging
            with open(f'error_page_{page}.html', 'w', encoding='utf-8') as f:
                f.write(driver.page_source)
            break

    with open(f"{group}.json", 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=4)
    print(f"Saved {len(all_results)} entries to {group}.json\n")

driver.quit()

NameError: name 'Options' is not defined

In [29]:
import csv
from collections import defaultdict

In [30]:
GROUP_STRUCTURE = {
    "summary": ["games", "goals", "expectedGoals", "successfulDribbles", "tackles", 
               "assists", "accuratePassesPercentage", "rating"],
    "attack": ["goals", "expectedGoals", "bigChancesMissed", "successfulDribbles",
              "totalShots", "goalConversionPercentage", "rating"],
    "defence": ["tackles", "interceptions", "clearances", "errorLeadToGoal", "rating"],
    "passing": ["bigChancesCreated", "assists", "accuratePasses", 
                "accuratePassesPercentage", "keyPasses", "rating"],
    "goalkeeper": ["saves", "cleanSheet", "penaltySave", "savedShotsFromInsideTheBox",
                  "runsOut", "rating"]
}

ordered_stats = []
seen_stats = set()
for group in GROUP_STRUCTURE.values():
    for stat in group:
        if stat == 'rating':
            continue
        if stat not in seen_stats:
            ordered_stats.append(stat)
            seen_stats.add(stat)

# CSV column order
fieldnames = ['player.name', 'player.id', 'team.name'] + ordered_stats + ['rating']

players = defaultdict(dict)

# pimport pandas as pdrocess each JSON file
for group in GROUP_STRUCTURE:
    try:
        with open(f"{group}.json", "r", encoding="utf-8") as f:
            data = json.load(f)
            
        for entry in data:
            player_id = entry["player"]["id"]
            
            for stat in GROUP_STRUCTURE[group]:
                value = entry.get(stat)
                if value is not None: 
                    players[player_id][stat] = value
                    
            # Always update name and team
            players[player_id]["player.name"] = entry["player"]["name"]
            players[player_id]["player.id"] = player_id
            players[player_id]["team.name"] = entry["team"]["name"]
            
    except FileNotFoundError:
        print(f"Warning: {group}.json not found")
        continue

sorted_players = sorted(players.values(), 
                       key=lambda x: x.get("rating", 0), 
                       reverse=True)

#write csv
with open("player_stats.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for player in sorted_players:
        row = {col: player.get(col) for col in fieldnames}
        writer.writerow(row)

print("CSV file created successfully: player_stats.csv")

NameError: name 'json' is not defined

In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('player_stats.csv')

#print(df.head())

cleaned_df = df.copy()

cleaned_df["expectedGoals"] = cleaned_df["expectedGoals"].fillna(0)

print(cleaned_df.head())
print("total NaN:")
print(cleaned_df.isna().sum()) 

       player.name  player.id          team.name  goals  expectedGoals  \
0            Rodri     827606    Manchester City      8           4.07   
1   Arijanet Murić     888971            Burnley      0           0.00   
2  Kevin De Bruyne      70996    Manchester City      4           2.41   
3       Phil Foden     859765    Manchester City     19          10.34   
4  Bruno Fernandes     288205  Manchester United     10          10.01   

   successfulDribbles  tackles  assists  accuratePassesPercentage  \
0                  42       70        9                     92.46   
1                   0        1        0                     67.49   
2                  10       14       10                     83.83   
3                  47       31        8                     89.10   
4                  19       68        8                     79.38   

   bigChancesMissed  ...  errorLeadToGoal  bigChancesCreated  accuratePasses  \
0                 2  ...                1                  8

In [11]:
import sqlite3

def initialize_sofascore_database():
    conn = sqlite3.connect('sofascore_player_analysis.db')
    cursor = conn.cursor()

    # create the players table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS players (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            team_name TEXT
        )
    ''')

    # creating player_statistics table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS player_statistics (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            player_id INTEGER NOT NULL,
            stat_name TEXT NOT NULL,
            stat_value REAL,
            FOREIGN KEY (player_id) REFERENCES players(id)
        )
    ''')

    conn.commit()
    conn.close()
    print("Database 'sofascore_player_analysis.db' and tables have been initialized.")


initialize_sofascore_database()

Database 'sofascore_player_analysis.db' and tables have been initialized.


In [12]:
import sqlite3
import pandas as pd

def insert_player_data_from_csv(csv_file):
    conn = sqlite3.connect('sofascore_player_analysis.db')
    cursor = conn.cursor()

    # fill missing values with 0
    df = pd.read_csv(csv_file)
    df = df.fillna(0)  
    
    for _, row in df.iterrows():
        player_id = row['player.id']
        name = row['player.name']
        team_name = row['team.name']

        # insert data into players table
        cursor.execute('''
            INSERT OR IGNORE INTO players (id, name, team_name)
            VALUES (?, ?, ?)
        ''', (player_id, name, team_name))

        #insert each stat from the row(exclude player and team info)
        for col in df.columns:
            if col in ['player.id', 'player.name', 'team.name']:
                continue

            stat_name = col
            stat_value = row[col]

            cursor.execute('''
                INSERT INTO player_statistics (player_id, stat_name, stat_value)
                VALUES (?, ?, ?)
            ''', (player_id, stat_name, stat_value))

    conn.commit()
    conn.close()
    print("Player and statistics data inserted into the database.")

# run funtion and insert data into databade
insert_player_data_from_csv("player_stats.csv")


Player and statistics data inserted into the database.


In [33]:
import sqlite3

def query_player_stats(player_name, top_n=None):
    conn = sqlite3.connect('sofascore_player_analysis.db')
    cursor = conn.cursor()

    # find id of player
    cursor.execute('SELECT id, name, team_name FROM players WHERE name = ?', (player_name,))
    result = cursor.fetchone()

    if not result:
        print(f"No player found with name '{player_name}'.")
        conn.close()
        return

    player_id, name, team = result
    print(f"\nStats for {name} ({team}):")

    # fetch all stats for the player
    cursor.execute('''
        SELECT stat_name, stat_value
        FROM player_statistics
        WHERE player_id = ?
    ''', (player_id,))
    
    stats = cursor.fetchall()
    if top_n is not None:
        stats = sorted(stats, key=lambda x: x[1], reverse=True)[:top_n]
        
    for stat_name, stat_value in stats:
        print(f"{stat_name}: {stat_value:.3f}")

    conn.close()

# example:
query_player_stats("Bukayo Saka", top_n=10)



Stats for Bukayo Saka (Arsenal):
accuratePasses: 1087.000
totalShots: 108.000
keyPasses: 91.000
accuratePassesPercentage: 82.980
tackles: 66.000
successfulDribbles: 49.000
clearances: 19.000
goals: 16.000
expectedGoals: 15.500
interceptions: 15.000


In [34]:
import sqlite3

def clean_sofascore_database():
    conn = sqlite3.connect('sofascore_player_analysis.db')
    cursor = conn.cursor()

    # remove player_stat rows with nulll stat_name or stat_value
    cursor.execute('''
        DELETE FROM player_statistics
        WHERE stat_name IS NULL OR stat_value IS NULL
    ''')

    # remove stats where value = 0 (for specific stats only)
    cursor.execute('''
        DELETE FROM player_statistics
        WHERE stat_name IN ('goals', 'expectedGoals', 'assists') AND stat_value = 0
    ''')

    conn.commit()
    conn.close()
    print("Database cleaned of NULLs and zero-valued goal-related stats.")


In [35]:
import shutil

def backup_sofascore_database():
    source = 'sofascore_player_analysis.db'
    backup = 'sofascore_player_analysis_backup.db'
    shutil.copy(source, backup)
    print(f"Backup created: {backup}")


In [36]:
def perform_sofascore_maintenance():
    print("Starting database maintenance...")
    clean_sofascore_database()
    backup_sofascore_database()
    print("Maintenance complete.")

# run maintnance
perform_sofascore_maintenance()


Starting database maintenance...
Database cleaned of NULLs and zero-valued goal-related stats.
Backup created: sofascore_player_analysis_backup.db
Maintenance complete.
